# 1. Neural Decision Variables Extraction

## 1.1 Project Context
This notebook implements the decision variables extraction methodology for the ClickDV project - a computational neuroscience analysis linking Poisson click input data from rats to decision variable outputs. This analysis is part of the Brody-Daw lab rotation work.

## 1.2 Objective
Extract decision variables (DVs) from neural spike data using logistic regression to decode choice from population firing rates. The methodology follows the approach described in "Brain-wide coordination of decision formation and commitment" (Bondy et al., 2025 Draft).

### 1.2.1 What are Decision Variables?

- **Definition**: 1-dimensional time series DV(t,i) representing neural population "confidence" in making a particular choice on trial i at time t
- **Interpretation**: Positive values indicate evidence for one choice (e.g., rightward), negative values for the alternative choice (e.g., leftward)
- **Units**: Log-odds of choice probability, equivalent to log(p_right / p_left)
- **Purpose**: Dimensionality reduction of hundreds of neurons into single time-varying decision signal

### 1.2.2 Key Methodology Features

- Logistic regression with L1 regularization
- 10-fold stratified cross-validation
- Geometric mean regularization approach for temporal consistency
- 50ms Gaussian smoothing of firing rates
- Trial-by-trial resolution capturing decision formation dynamics

## 1.3 Setup and Dependencies

### 1.3.1 Import Required Libraries

In [ ]:
# Core scientific computing
import numpy as np
import scipy as sp
from scipy import ndimage
from scipy.io import loadmat
import pandas as pd

# Machine learning
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 11
})

# Suppress sklearn warnings
warnings.filterwarnings('ignore', category=UserWarning)

print("Libraries imported successfully")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {sp.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

### 1.3.2 Set Reproducibility Parameters

In [ ]:
# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Analysis parameters (from paper methodology)
GAUSSIAN_SIGMA_MS = 50  # Gaussian smoothing kernel standard deviation (ms)
SAMPLING_INTERVAL_MS = 50  # Time bin size (ms)
TIME_WINDOW = (-0.5, 1.5)  # Analysis window relative to stimulus onset (seconds)
CV_FOLDS = 10  # Number of cross-validation folds

# Session quality control criteria (from Methods, page 30)
MIN_TRIALS = 300  # Minimum trials required for session inclusion
MAX_LAPSE_RATE = 0.08  # Maximum allowed lapse rate (8%)

print(f"Random seed set to: {RANDOM_SEED}")
print(f"Gaussian smoothing: {GAUSSIAN_SIGMA_MS}ms standard deviation")
print(f"Sampling interval: {SAMPLING_INTERVAL_MS}ms")
print(f"Analysis window: {TIME_WINDOW[0]}s to {TIME_WINDOW[1]}s")
print(f"Cross-validation: {CV_FOLDS} folds")
print(f"Quality control: ≥{MIN_TRIALS} trials, ≤{MAX_LAPSE_RATE*100}% lapse rate")

### 1.3.3 Define File Paths and Session Parameters

In [ ]:
# Data paths
PROJECT_ROOT = Path('/home/ham/SF/Personal/Education/07 Princeton/Rotations/Brody-Daw/ClickDV')
DATA_DIR = PROJECT_ROOT / 'data' / 'raw' / 'A324' / '2023-07-27'
OUTPUT_DIR = PROJECT_ROOT / 'notebooks' / 'analysis' / 'outputs'

# Session-specific parameters
SESSION_ID = 'A324'
SESSION_DATE = '2023-07-27'
DATA_FILE = DATA_DIR / 'A324_pycells_20230727.mat'

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Verify data file exists
if not DATA_FILE.exists():
    raise FileNotFoundError(f"Data file not found: {DATA_FILE}")

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Session: {SESSION_ID} ({SESSION_DATE})")
print(f"Data file: {DATA_FILE}")
print(f"Data file exists: {DATA_FILE.exists()}")
print(f"Data file size: {DATA_FILE.stat().st_size / 1024 / 1024:.1f} MB" if DATA_FILE.exists() else "N/A")

### 1.3.4 Define Core Analysis Functions

These function signatures will be implemented in subsequent sections of the notebook.

In [ ]:
# Function signatures for the complete pipeline
# These will be implemented in subsequent notebook sections

def calculate_firing_rates(spike_times: Dict[str, np.ndarray], 
                          trial_times: np.ndarray, 
                          time_bins: np.ndarray, 
                          sigma_ms: float = GAUSSIAN_SIGMA_MS) -> np.ndarray:
    """
    Calculate smoothed firing rates using Gaussian convolution.
    
    Parameters:
    - spike_times: Dict of {neuron_id: spike_times_array}
    - trial_times: Array of trial start times
    - time_bins: Time points relative to trial start
    - sigma_ms: Gaussian kernel standard deviation (ms)
    
    Returns:
    - firing_rates: Array of shape (n_neurons, n_timepoints, n_trials)
    """
    pass  # Implementation in Section 3

def apply_quality_filters(spike_times: Dict[str, np.ndarray], 
                         quality_metrics: Dict[str, Dict]) -> Dict[str, np.ndarray]:
    """
    Apply neuron quality filters based on waveform characteristics.
    
    Parameters:
    - spike_times: Dict of {neuron_id: spike_times_array}
    - quality_metrics: Dict of quality metrics for each neuron
    
    Returns:
    - filtered_spike_times: Dict with only high-quality neurons
    """
    pass  # Implementation in Section 3

def validate_session_quality(choices: np.ndarray, 
                           n_trials: int, 
                           lapse_rate: Optional[float] = None) -> Tuple[bool, str]:
    """
    Validate session meets quality criteria from paper.
    
    Parameters:
    - choices: Array of behavioral choices
    - n_trials: Number of trials
    - lapse_rate: Session lapse rate (optional)
    
    Returns:
    - (passes_qc, reason): Boolean and descriptive string
    """
    pass  # Implementation in Section 4

def find_optimal_regularization(X: np.ndarray, 
                               choices: np.ndarray) -> Tuple[float, List[float]]:
    """
    Find optimal regularization using geometric mean approach.
    
    Parameters:
    - X: Neural data (n_neurons, n_timepoints, n_trials)
    - choices: Binary choices (n_trials,)
    
    Returns:
    - (final_C, individual_lambdas): Optimal regularization parameter and individual values
    """
    pass  # Implementation in Section 5

def calculate_decision_variables(X: np.ndarray, 
                               choices: np.ndarray, 
                               final_C: float) -> Tuple[np.ndarray, List, np.ndarray]:
    """
    Calculate decision variables using logistic regression.
    
    Parameters:
    - X: Neural data (n_neurons, n_timepoints, n_trials)
    - choices: Binary choices (n_trials,)
    - final_C: Regularization parameter
    
    Returns:
    - (DVs, models, accuracies): Decision variables, fitted models, and accuracies
    """
    pass  # Implementation in Section 6

print("Function signatures defined")
print("Ready to proceed with data loading and analysis")
print("\nNext steps:")
print("1. Load and explore data structure (Section 2)")
print("2. Implement data preprocessing (Section 3)")
print("3. Apply session quality control (Section 4)")
print("4. Implement model training and cross-validation (Section 5)")
print("5. Calculate decision variables (Section 6)")
print("6. Create visualizations and validation (Sections 7-8)")
print("7. Export results and summary (Section 9)")

---
## Setup Complete

This completes the **Introduction & Setup** section of the decision variables extraction notebook. The environment is now configured with:

✅ **Libraries imported**: NumPy, SciPy, scikit-learn, Matplotlib, Seaborn  
✅ **Reproducibility ensured**: Random seed set to 42  
✅ **Parameters configured**: Following paper methodology (50ms smoothing, 50ms sampling, 10-fold CV)  
✅ **File paths defined**: Data source and output directories established  
✅ **Function signatures**: Complete pipeline structure outlined  

**Session Details:**
- **Target**: A324 session from 2023-07-27
- **Data file**: A324_pycells_20230727.mat 
- **Analysis window**: -500ms to +1500ms relative to stimulus onset
- **Quality criteria**: ≥300 trials, ≤8% lapse rate

The notebook is ready for **Section 2: Data Loading & Exploration**.